In [1]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np
import pandas as pd

import pickle
import jieba
import h5py

SOS_token = 0
EOS_token = 1

# dataProcess

In [2]:
class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    #通过不断输入sentence（字符串的格式），构建词与下标的对应（词典），方便制作one-hot。
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            
    def save(self, path):
        with open(path,'wb') as f:
            pickle.dump([self.name,self.word2index, self.word2count, self.index2word, self.n_words],f)
    
    def load(self,path):
        with open(path,'rb') as f:
            name, self.word2index, self.word2count, self.index2word, self.n_words = pickle.load(f)
        if self.name != name:
            print('error: Name error------------------------------!')
            
            
##################################################################

def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def normalizeChinese(s):
    try:
        s.encode("gb2312")
    except UnicodeEncodeError:
        return ' '
    s = re.sub(r"[~!@#$%^&* ]+",r' ', s)
    return s


#lang1 = 'zh'  lang2 = 'en'
#默认英文到中文
def readTrainLangs(lang1, lang2, reverse=True,fenci = False):
    print("Reading lines...")

    zh_lines = open('../data/train.%s'% lang1).read().strip().split('\n')
    #zh_lines = zh_lines[0:20]  #for test

    zh_data_list = []
    if fenci:
        #jieba 分词
        for line in zh_lines:
            seg_line = jieba.cut(line,cut_all=False)
            #dic = [seg for seg in seg_line]
            dic = ' '.join(seg_line)
            tmp = ' '
            for char in dic.split(' '):
                val = normalizeChinese(char)
                tmp += val+' '
            zh_data_list.append(tmp)
    else: #用空格按字分开
        for line in zh_lines:
            dic = ' '.join(line)
            tmp = ' '
            for char in dic.split(' '):
                val = normalizeChinese(char)##去除生僻词
                tmp += val+' '
            zh_data_list.append(tmp)

    en_lines = open('../data/train.%s'% lang2).read().strip().split('\n')
    #en_lines = en_lines[0:20]  #for test
    
    # Split every line into pairs and normalize
    #去掉一些标点符号
    en_data_list = [[normalizeString(s) for s in l.split('\t')] for l in en_lines]
    pairs = []
    if reverse:
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
        for en,zh in zip(en_data_list,zh_data_list):
            input_lang.addSentence(en[0])
            output_lang.addSentence(zh)
            pairs.append([en[0].encode('utf-8'),zh.encode('gb2312')])
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
        for en,zh in zip(en_data_list,zh_data_list):
            input_lang.addSentence(zh)
            output_lang.addSentence(en[0])
            pairs.append([zh.encode('gb2312'), en[0].encode('utf-8')])
            
    return input_lang, output_lang, pairs
##################################################

#这部分就是对数据进行处理的函数了，上面写的函数都会在这里被调用
#最后得到三个变量input_lang，output_lang分别是源语言和目标语言的类，包含它们各自的词典。
#pairs是一个列表，列表的元素是一个二元tuple，tuple里面的内容是一句源语言字符串，一句目标语言字符串。
def prepareData(lang1, lang2, reverse=True, fenci=False):
    input_lang, output_lang, pairs = readTrainLangs(lang1, lang2, reverse, fenci)
    print("Read %s sentence pairs" % len(pairs))
    print(pairs[0][0].decode('utf-8'),pairs[0][1].decode('gb2312'))
    #pairs = filterPairs(pairs)
    #print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
#     for pair in pairs:
#         input_lang.addSentence(pair[0].decode('utf-8'))
#         output_lang.addSentence(pair[1].decode('gb2312'))
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


In [ ]:
inputLang, outputLang, pairs = prepareData('zh','en')

## save data

In [ ]:
inputLang.save('../data/en_train.pkl')
outputLang.save('../data/zh_train.pkl')

h5 = h5py.File('../data/train_afterProcess.h5py','w')
h5.create_dataset('pairs',data=pairs,dtype = 'S400')
h5.close()

### load data

In [3]:
import dataProcess as dp
h5py_file = h5py.File('../data/train_afterProcess.h5py','r')
pairs = h5py_file['pairs']

pairs = pairs[0:1000]
print(pairs[0][0].decode('utf-8'))
print(pairs[0][1].decode('gb2312'))

inputlang = dp.Lang('en')
outputlang = dp.Lang('zh')
# inputlang.load('../data/en_train.pkl')
# outputlang.load('../data/zh_train.pkl')

####测试用，上面两行注释的语句在真正运行的时候要用到的##################################################################
for pair in pairs:
    inputlang.addSentence(pair[0].decode('utf-8'))
    outputlang.addSentence(pair[1].decode('gb2312'))
    
print(inputlang.name,inputlang.n_words)
print(outputlang.name,outputlang.n_words)
#h5py_file.close()


a pair of red crowned cranes have staked out their nesting territory
 一 对 丹 顶 鹤 正 监 视 着 它 们 的 筑 巢 领 地 
en 1842
zh 1408


10000


### filter -- remove words that appear less than 10 times

In [4]:
print(len(inputlang.word2count))
print(len(inputlang.word2index))
print(len(inputlang.index2word))
all_en_words = inputlang.word2count.copy()
for word in all_en_words:
    if  all_en_words[word] <= 10:
        inputlang.word2count.pop(word)
        index = inputlang.word2index[word]
        inputlang.word2index.pop(word)
        inputlang.index2word.pop(index)
        
print(len(inputlang.word2count))
print(len(inputlang.word2index))
print(len(inputlang.index2word))

388089
388089
388091
82101
82101
82103


# build model 

In [22]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F

from hyperboard import Agent

agent = Agent(address='127.0.0.1',port=5000)
#agent = Agent(address='172.18.216.69',port=5000)
hyperparameters = {'learning rate':0.01}
name = agent.register(hyperparameters, 'loss',overwrite=True)

use_cuda = torch.cuda.is_available()
MAX_LENGTH = 30
teacher_forcing_ratio = 1  #在训练时解码器使用labels（平行预料）进行训练的概率
LEARNING_RATE = 0.01


In [5]:
class EncoderRNN(nn.Module):
    #input_size是指词典的大小(毕竟要建立embedding)，hidden_size是hidden_state的维度
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    #input是一个句子(这个句子已经通过数据处理的类转换成下标，这样可以对应一个embedded)
    #hidden 是上一个迭代中的hidden，即pre_hidden
    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        #这个n_layers==1其实就是只相当于一个cell，对一个input(单词)和上一个hidden state
        #这里做了一个gru操作。n_layers大于1则是对同一个东西迭代多次，也许效果会好。
        for i in range(self.n_layers):
            output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result
        
#hidden_size都是说hidden_state的维度，要和encoder一致。
#output_size是目标语言的词典大小，因为输出的是所有词的概率
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1):
        super(DecoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        for i in range(self.n_layers):
            output = F.relu(output)
            output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result
        
#hidden_size都是说hidden_state的维度，要和encoder一致。
#output_size是目标语言的词典大小，因为输出的是所有词的概率
#max_length是句子的最大长度(之前被限制了，以后看看能否不要这个限制)
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    #input是一个目标句子中的某个词(这个词已经通过数据处理的类转换成下标，这样可以对应一个embedded)。
    #当然，在进行预测的时候就不会是输入目标句子的词了。而是它预测出来的词
    #hidden 是上一个迭代中的hidden，即pre_hidden
    #encoder_output是encoder最后一个输出，不过在这里它并没有被使用到
    #encoder_outputs是encoder每次输出(y1,y2,...,yn)的组成tensor，格式跟input一样，只不过它是句子，而不是某个词。
    def forward(self, input, hidden, encoder_output, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)))
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        for i in range(self.n_layers):
            output = F.relu(output)
            output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]))#每个词的概率
        return output, hidden, attn_weights

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

# training

### 将数据进行处理成pytorch变量，方便转换成embedded

In [6]:
#########################training#####################################
#返回词对应的下标
def indexesFromSentence(lang, sentence):
    #return [lang.word2index[word] for word in sentence.split(' ')]
    result = []
    all_lang_keys = lang.word2index.keys()
    for word in sentence.split(' '):
        if word in all_lang_keys:#判断词是否在词典中，因为词典中有些出现次数太少的词被删掉了
            result.append(lang.word2index[word])
    return result

#将词转换成variable
def variableFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    result = Variable(torch.LongTensor(indexes).view(-1, 1))
    if use_cuda:
        return result.cuda()
    else:
        return result

    
def variablesFromPair(pair,input_lang, output_lang):
    #注意这里要先解码，因为保存到h5py里面的时候要编码，所以现在要解码
    input_variable = variableFromSentence(input_lang, pair[0].decode('utf-8'))
    target_variable = variableFromSentence(output_lang, pair[1].decode('gb2312'))
    return (input_variable, target_variable)




### 定义训练所需要的函数

In [7]:
def train(input_variable, target_variable, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    #input_variable已经在函数外变成了tensor了，tensor的元素是词的下标
    input_length = input_variable.size()[0]#source sentence 的长度
    target_length = target_variable.size()[0]#目标句子的长度

    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))#max_length=10，也就是句子的最长长度，hidden_size是256，所以encoder_outputs是矩阵10X256
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

    loss = 0

    for ei in range(input_length):#句子有多长就迭代多少次
        encoder_output, encoder_hidden = encoder(
            input_variable[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0][0]#将每个词encoder的output记录下来

    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    decoder_hidden = encoder_hidden#encoder最后一层的hidden_state传给decoder作为decoder的第一个hidden_state

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
        	#encoder_outputs作为decoder的输入，是为了改变attention。
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_output, encoder_outputs)
            loss += criterion(decoder_output, target_variable[di])#这两个变量是什么形式
            decoder_input = target_variable[di]  # Teacher forcing这个是直接给答案，也就是一个单词，进入decoder里面再变成词向量

    else:#这边是不直接给答案，而是每次output那里选择概率最大的作为下一个输入
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_output, encoder_outputs)
            #从decoder的类中可以知道，decoder_output是softmax出来的，即所有词的概率。
            #topk函数是查找最大的K 个数，这里参数是1，topv就是value，topi是index，也就是词对应的下标
            topv, topi = decoder_output.data.topk(1)
            ni = topi[0][0]

            decoder_input = Variable(torch.LongTensor([[ni]]))
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input

            loss += criterion(decoder_output, target_variable[di])
            if ni == EOS_token:
                break

    loss.backward()#如何理解这一步反向梯度对encoder和decoder都有效

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.data[0] / target_length



In [19]:
######################################################################
# This is a helper function to print time elapsed and estimated time
# remaining given the current time and progress %.
#

import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent) #总时间
    rs = es - s        #总时间减去已经运行的时间等于还剩下的时间
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

######################################################################
# The whole training process looks like this:
#
# -  Start a timer
# -  Initialize optimizers and criterion
# -  Create set of training pairs
# -  Start empty losses array for plotting
#
# Then we call ``train`` many times and occasionally print the progress (%
# of examples, time so far, estimated time) and average loss.
#

def trainIters(encoder, decoder, inputlang, outputlang, n_iters, print_every=1000, plot_every=100, learning_rate=0.01, save_model_every=10000):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    #考虑改成其他的优化器
    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    #这里的training_pairs经过variableFromPair处理后，每个元素已经是一个tensor了，并且是单词所在的下标，为了可以和embedd匹配。
    #training_pairs = [variablesFromPair(random.choice(pairs))
    #                 for i in range(n_iters)]
    #print(random.choice(training_pairs)[0].data)
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        #training_pair = training_pairs[iter - 1]
        #################@#%…………&&&
        
        training_pair = variablesFromPair(random.choice(pairs),inputlang,outputlang)
        input_variable = training_pair[0]
        target_variable = training_pair[1]

        loss = train(input_variable, target_variable, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            agent.append(name, iter, plot_loss_avg)
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0
            
        if iter % save_model_every == 0:
            torch.save(encoder.state_dict(),'../models/encoder.model{0}'.format(iter))
            torch.save(decoder.state_dict(),'../models/decoder.model{0}'.format(iter))
            pass


### real training

In [23]:
hidden_size = 256
encoder1 = EncoderRNN(inputlang.n_words, hidden_size)
attn_decoder1 = AttnDecoderRNN(hidden_size, outputlang.n_words,
                               1, dropout_p=0.1)

if use_cuda:
    encoder1 = encoder1.cuda()
    attn_decoder1 = attn_decoder1.cuda()

trainIters(encoder1, attn_decoder1, inputlang, outputlang, 80000, print_every=100, save_model_every=2000)

######################################################################

0m 6s (- 90m 12s) (100 0%) 4.6545
0m 13s (- 89m 25s) (200 0%) 4.0698
0m 21s (- 94m 39s) (300 0%) 3.8483
0m 28s (- 96m 7s) (400 0%) 3.7090
0m 37s (- 100m 30s) (500 0%) 3.6727
0m 46s (- 103m 39s) (600 0%) 3.7431
0m 54s (- 102m 55s) (700 0%) 3.5381
1m 2s (- 102m 57s) (800 1%) 3.4063
1m 10s (- 103m 4s) (900 1%) 3.3690
1m 18s (- 102m 56s) (1000 1%) 3.3279
1m 26s (- 102m 56s) (1100 1%) 3.2043
1m 34s (- 103m 18s) (1200 1%) 3.2180
1m 42s (- 103m 16s) (1300 1%) 3.3067
1m 49s (- 102m 22s) (1400 1%) 3.1445
1m 56s (- 101m 34s) (1500 1%) 3.1311
2m 3s (- 101m 14s) (1600 2%) 3.1672
2m 11s (- 100m 52s) (1700 2%) 2.9504
2m 19s (- 101m 3s) (1800 2%) 3.1100
2m 27s (- 101m 20s) (1900 2%) 2.9769
2m 37s (- 102m 3s) (2000 2%) 3.1005
2m 45s (- 102m 9s) (2100 2%) 2.8924
2m 53s (- 102m 8s) (2200 2%) 2.8906
3m 1s (- 101m 58s) (2300 2%) 2.8482
3m 9s (- 102m 4s) (2400 3%) 2.9255
3m 17s (- 101m 58s) (2500 3%) 2.8514
3m 24s (- 101m 29s) (2600 3%) 2.6627
3m 32s (- 101m 21s) (2700 3%) 2.6850
3m 40s (- 101m 17s) (2800 

SystemExit: 

/data/liwb/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


# 评估

In [24]:
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np

######################################################################
# Evaluation
# ==========
#
# Evaluation is mostly the same as training, but there are no targets so
# we simply feed the decoder's predictions back to itself for each step.
# Every time it predicts a word we add it to the output string, and if it
# predicts the EOS token we stop there. We also store the decoder's
# attention outputs for display later.
#

def evaluate(encoder, decoder, sentence, input_lang, output_lang, max_length=MAX_LENGTH):
    input_variable = variableFromSentence(input_lang, sentence)
    input_length = input_variable.size()[0]
    encoder_hidden = encoder.initHidden()

    encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
    encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_variable[ei],
                                                 encoder_hidden)
        encoder_outputs[ei] = encoder_outputs[ei] + encoder_output[0][0]

    decoder_input = Variable(torch.LongTensor([[SOS_token]]))  # SOS
    decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    decoder_hidden = encoder_hidden

    decoded_words = []
    decoder_attentions = torch.zeros(max_length, max_length)

    for di in range(max_length):#注意这里跟训练的时候不一样，训练的时候用的是target_length。这里因为要输出句子，而代码限定了句子的最大长度。
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_output, encoder_outputs)
        decoder_attentions[di] = decoder_attention.data
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        if ni == EOS_token:
            decoded_words.append('<EOS>')  #检测到结束符就停止
            break
        else:
            decoded_words.append(output_lang.index2word[ni])

        decoder_input = Variable(torch.LongTensor([[ni]]))
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input

    return decoded_words, decoder_attentions[:di + 1]


In [25]:
######################################################################
# We can evaluate random sentences from the training set and print out the
# input, target, and output to make some subjective quality judgements:
#

def evaluateRandomly(encoder, decoder, inputlang, outputlang, n=100):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0].decode('utf-8'))
        print('=', pair[1].decode('gb2312'))
        output_words, attentions = evaluate(encoder, decoder, pair[0].decode('utf-8'),inputlang, outputlang)
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [26]:
evaluateRandomly(encoder1,attn_decoder1,inputlang,outputlang)

> a slight form of turbulence .
=  一 小 团 空 气 流 。 
<  一 小 团 空 气 流 。  <EOS>

> a letter for you madam .
=  一 封 你 的 信 ， 女 士 。 
<  一 封 你 的 信 ， 女 士 。  <EOS>

> i want tact ops on site within the hour .
=  一 小 时 内 建 好 战 略 指 挥 部 。 
<  一 小 时 内 建 好 战 略 指 挥 部 。  <EOS>

> one couple was white the other black .
=  一 对 是 白 人 一 对 是 黑 人 。 
<  一 对 是 白 人 一 对 是 黑 人 。  <EOS>

> i ll have your warrant in an hour .
=  一 小 时 之 内 给 你 搜 查 令 。 
<  一 小 时 之 内 给 你 搜 查 令 。  <EOS>

> four inches of rain in under an hour .
=  一 小 时 之 内 下 了 4 寸 的 雨 量 
<  一 小 时 之 内 下 了 4 寸 的 雨 量  <EOS>

> couple comes by .
=  一 对 情 侣 经 过 。 
<  一 对 情 侣 经 过 。  <EOS>

> your father passed away an hour ago .
=  一 小 时 前 你 父 亲 过 世 了 。 
<  一 小 时 前 你 父 亲 过 世 了 。  <EOS>

> some newlyweds was killed . some more of it he said .
=  一 对 新 婚 夫 妇 被 杀 了 。 他 说 还 有 更 多 会 被 杀 。 
<  一 对 新 婚 夫 妇 被 杀 了 。 他 说 还 有 更 多 会 被 杀 。  <EOS>

> a piece of cake isn t gonna hurt those .
=  一 小 块 蛋 糕 没 问 题 的 。 
<  一 小 块 蛋 糕 没 问 题 的 。  <EOS>

> a ietter to myseif to be s